# 数据准备部分

In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
import re
import gc
from tqdm import tqdm

In [2]:
age_test_data = pd.read_csv(r'../data/age_test.csv',header=None,names=['uId'])
age_train_data = pd.read_csv( r'../data/age_train.csv',header=None,names=['uId','age_group'])
age = age_train_data['age_group']
age_train_data.drop(columns=['age_group'],inplace=True)
print(age_train_data.shape)     #(2010000, 1)
print(age_test_data.shape)

(2010000, 1)
(502500, 1)


In [3]:
user_app_actived_data = pd.read_csv(r'../data/user_app_actived.csv',header=None,names=['uId','appId'])
print(user_app_actived_data.head());print(user_app_actived_data.shape)

       uId                                              appId
0  1000008  a00105157#a00109386#a00134746#a0013952#a001451...
1  1000010  a00102782#a00107077#a00109386#a00134746#a00136...
2  1000011  a00158535#a00163116#a00170432#a00187480#a00224...
3  1000013  a00109386#a0011601#a0012768#a00136287#a0014516...
4  1000014  a00109386#a0011894#a0013059#a00134840#a0013578...
(2512500, 2)


In [4]:
user_numAppList = {} # {uId:[num,appList]}
for user in tqdm(user_app_actived_data.values):
    appList = re.split(r'\#',user[1])
    if appList[0] == '\\N':
        user_numAppList[user[0]] = []
        l = 0
    else:
        user_numAppList[user[0]] = appList
        l = len(appList)
    user_numAppList[user[0]].insert(0,l)
# print(user_numAppList[1002181])
# print(user_numAppList[1002179])

100%|██████████| 2512500/2512500 [01:03<00:00, 39287.42it/s] 


In [5]:
del user_app_actived_data
gc.collect()

7

In [6]:
app_info_data = pd.read_csv(r'../data/app_info.csv',header=None,names=['appId','category'])
data = app_info_data['category'].value_counts(dropna=False)
print(data.index)
print(data.values)
app_category = {} #{appId:[category]}
for aid in app_info_data.values:
    app_category.setdefault(aid[0],[]).append(aid[1])
print('总共有多少个app:   ',len(app_category))

Index(['实用工具', '便捷生活', '教育', '金融理财', '购物比价', '社交通讯', '影音娱乐', '新闻阅读', '休闲益智',
       '商务', '运动健康', '出行导航', '经营策略', '动作射击', '儿童', '角色扮演', '拍摄美化', '棋牌桌游',
       '旅游住宿', '汽车', '主题个性', '美食', '体育竞速', '网络游戏', '休闲游戏', '动作冒险', '休闲娱乐',
       '学习办公', '表盘个性', '益智棋牌', '策略游戏', '电子书籍', '模拟游戏', '图书阅读', '棋牌天地', '体育射击',
       '角色游戏', '主题铃声', '医疗健康', '合作壁纸*'],
      dtype='object')
[57608 20164 14348 11842 10400 10048  7996  7946  7862  6237  5139  3712
  3661  3448  3246  3196  3176  2876  1524  1315  1091  1003   789   128
    31    10    10     9     6     6     5     5     5     4     4     4
     3     3     2     2]
总共有多少个app:    167622


In [7]:
category_num = {}
def initDict(categorylist):
    for cate in categorylist:
        category_num[cate] = 0
    return category_num
categorylist = data.index
category_num = initDict(categorylist)
print(category_num.values());print(categorylist)

dict_values([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Index(['实用工具', '便捷生活', '教育', '金融理财', '购物比价', '社交通讯', '影音娱乐', '新闻阅读', '休闲益智',
       '商务', '运动健康', '出行导航', '经营策略', '动作射击', '儿童', '角色扮演', '拍摄美化', '棋牌桌游',
       '旅游住宿', '汽车', '主题个性', '美食', '体育竞速', '网络游戏', '休闲游戏', '动作冒险', '休闲娱乐',
       '学习办公', '表盘个性', '益智棋牌', '策略游戏', '电子书籍', '模拟游戏', '图书阅读', '棋牌天地', '体育射击',
       '角色游戏', '主题铃声', '医疗健康', '合作壁纸*'],
      dtype='object')


In [8]:
cate = list(np.zeros(40))
i = 1
batch_size = 10000 #每10000次存储一次数据
for numAppList in tqdm(user_numAppList.values()):
#     print(i)
    i += 1
    categorylist = data.index
    category_num = initDict(categorylist)
    if len(numAppList)==1:
        cate = np.vstack((cate,list(category_num.values())))
    else:
        appList = numAppList[1:-1]
        for appId in appList:
            if appId in app_category:
                cateList = app_category[appId]
                for category in cateList:
                    category_num[category] += 1
        cate = np.vstack((cate,list(category_num.values())))
    if i%batch_size==0:
        if i==batch_size:
            cate = pd.DataFrame(cate, columns=['实用工具', '便捷生活', '教育', '金融理财', '购物比价', '社交通讯', '影音娱乐', '新闻阅读', '休闲益智',
                                                   '商务', '运动健康', '出行导航', '经营策略', '动作射击', '儿童', '角色扮演', '拍摄美化', '棋牌桌游',
                                                   '旅游住宿', '汽车', '主题个性', '美食', '体育竞速', '网络游戏', '休闲游戏', '休闲娱乐', '动作冒险',
                                                   '学习办公', '益智棋牌', '表盘个性', '电子书籍', '模拟游戏', '策略游戏', '棋牌天地', '体育射击',
                                                   '图书阅读','主题铃声', '角色游戏', '合作壁纸*', '医疗健康'])
        else:
            cate = pd.DataFrame(cate)
        cate.drop(0,inplace=True)
        if i==batch_size:
            cate.to_csv(r'../processed/category_num.csv', index=False, header=True, encoding='utf_8_sig',mode='a')
        else:
            cate.to_csv(r'../processed/category_num.csv', index=False, header=False, encoding='utf_8_sig', mode='a')
        cate = list(np.zeros(40))
cate = pd.DataFrame(cate)
cate.drop(0,inplace=True)
# print(cate);print(cate.head())
cate.to_csv(r'../processed/category_num.csv', index=False, header=False, encoding='utf_8_sig',mode='a')

100%|██████████| 2512500/2512500 [11:29<00:00, 3644.91it/s] 


        0     1     2     3     4     5     6     7    8     9  ...    30  \
1     16.0   8.0   8.0   1.0   1.0   5.0   7.0   1.0  0.0   1.0 ...   0.0   
2     11.0   7.0   6.0   2.0   3.0   3.0   5.0   2.0  0.0   1.0 ...   0.0   
3     13.0   5.0   2.0   1.0   0.0   1.0   1.0   1.0  1.0   1.0 ...   0.0   
4     23.0  10.0   4.0   2.0   6.0   7.0   6.0   6.0  3.0   1.0 ...   0.0   
5     43.0  18.0  23.0  13.0   9.0  10.0  15.0  13.0  1.0  17.0 ...   0.0   
6     31.0  20.0   7.0   4.0   6.0   9.0  13.0   7.0  0.0   2.0 ...   0.0   
7     15.0  10.0   5.0   3.0   8.0   6.0   6.0   5.0  0.0   2.0 ...   0.0   
8     13.0  11.0   5.0   1.0   6.0   4.0   5.0   3.0  0.0   5.0 ...   0.0   
9     10.0   8.0   3.0   2.0   4.0   8.0   5.0   2.0  0.0   1.0 ...   0.0   
10    15.0   6.0   3.0   4.0   8.0   4.0   8.0   2.0  1.0   2.0 ...   0.0   
11    14.0   8.0   3.0   0.0   7.0   4.0   6.0   2.0  0.0   4.0 ...   0.0   
12    17.0  10.0  17.0   1.0   5.0   8.0   9.0   4.0  1.0   3.0 ...   0.0   

In [9]:
del user_numAppList
gc.collect()

0

# 特征生成部分

In [3]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [4]:
category_num = pd.read_csv(r'../processed/category_num.csv')
print(category_num.shape)
# print(category_num.head())

(983611, 31)


In [12]:
col = pd.DataFrame(category_num.sum())
col.shape

(40, 1)

In [13]:
category_num = pd.DataFrame(category_num.values[:,col[0]>1000])
category_num.shape

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


(2512500, 30)

In [14]:
user_app_actived = pd.read_csv(r'../data/user_app_actived.csv',header=None)
print(user_app_actived.shape)
print(user_app_actived.head())

(2512500, 2)
         0                                                  1
0  1000008  a00105157#a00109386#a00134746#a0013952#a001451...
1  1000010  a00102782#a00107077#a00109386#a00134746#a00136...
2  1000011  a00158535#a00163116#a00170432#a00187480#a00224...
3  1000013  a00109386#a0011601#a0012768#a00136287#a0014516...
4  1000014  a00109386#a0011894#a0013059#a00134840#a0013578...


In [15]:
category_num = preprocessing.StandardScaler().fit_transform(category_num.values)
uId = user_app_actived[0].values

In [16]:
uId = uId.reshape(len(uId),1)
category_num.shape

(2512500, 30)

In [17]:
category_num = np.column_stack((uId,category_num))

In [18]:
category_num = pd.DataFrame(category_num)
print(category_num.shape)
print(category_num.head())

(2512500, 31)
          0         1         2         3         4         5         6   \
0  1000008.0 -0.334211  0.270043  0.282272  0.273748  1.060285  0.853186   
1  1000010.0  0.016666  0.935803 -0.071270 -0.173857  0.743862 -0.198733   
2  1000011.0 -1.269882 -1.061476 -1.131894 -0.845263 -0.521828 -0.549373   
3  1000013.0 -0.334211  0.270043 -0.424811  0.273748  0.743862  0.151907   
4  1000014.0  0.952337  0.769363  2.049980  0.049946  0.427440  0.151907   

         7         8         9     ...           21        22        23  \
0 -0.542339 -0.747547 -0.508995    ...    -0.382519  0.149207 -0.194018   
1 -0.298918  0.535790 -0.508995    ...    -0.382519 -0.587036 -0.194018   
2 -0.542339 -0.319768  0.448749    ...    -0.382519 -0.587036 -0.194018   
3 -0.542339  0.108011 -0.508995    ...    -0.382519 -0.587036 -0.194018   
4  0.431343  0.963569 -0.508995    ...    -0.382519 -0.587036 -0.194018   

         24        25        26        27       28        29        30  
0 -0.

In [19]:
age_train = pd.read_csv(r'../data/age_train.csv',header=None,usecols=[0])
print(age_train.shape)
age_test = pd.read_csv(r'../data/age_test.csv',header=None)
print(age_test.shape)

(2010000, 1)
(502500, 1)


In [20]:
age_train = pd.merge(age_train,category_num,how='inner',on=0)
# print(age_train.head(10))

        0         1         2         3         4         5         6   \
0  1000001 -0.100293 -0.562156 -1.131894 -0.621461 -1.471096 -0.900013   
1  1000011 -1.269882 -1.061476 -1.131894 -0.845263 -0.521828 -0.549373   
2  1000015 -0.802046 -0.562156  0.282272 -0.621461 -0.521828 -0.900013   
3  1000019 -1.737717 -1.560795 -1.485436 -0.845263 -1.471096 -1.601293   
4  1000023 -1.035964 -1.061476 -1.131894 -0.845263 -1.154674 -0.549373   
5  1000025  0.718419  0.769363  0.282272  1.168957 -0.521828  1.203826   
6  1000029  2.004967  1.768003  0.635814  0.273748  1.376708  0.853186   
7  1000035 -0.451170 -0.229276 -0.071270 -0.173857  0.111017 -0.198733   
8  1000037  1.420173  2.600202 -0.424811  1.168957  0.743862 -0.198733   
9  1000038 -0.451170 -0.229276 -0.778353 -0.845263  0.743862 -0.198733   

         7         8         9     ...           21        22        23  \
0 -1.029180 -1.175326 -0.508995    ...    -0.382519 -0.587036 -0.194018   
1 -0.542339 -0.319768  0.448749    

In [21]:
age_test = pd.merge(age_test,category_num,how='inner',on=0)
# print(age_test.head(10))

In [22]:
age_train.drop(labels=[0],axis=1,inplace=True)
age_test.drop(labels=[0],axis=1,inplace=True)

In [23]:
from scipy import sparse

In [24]:
age_train = sparse.csr_matrix(age_train,dtype=np.float32)
age_test = sparse.csr_matrix(age_test,dtype=np.float32)

In [25]:
print(age_train.shape)
print(age_test.shape)

(2010000, 30)
(502500, 30)


In [26]:
sparse.save_npz(r'../trainTestData/trainData30.npz',age_train)
sparse.save_npz(r'../trainTestData/testData30.npz',age_test)

In [ ]:
category_num.to_csv(r'../processed/category_num.csv',header=None,index=False)